<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_premade_estimators_new_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0-rc2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc2
tensorflow.keras 2.3.0-tf


In [4]:
#如何在colab上使用在google cloud上面的文件
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/Colab Notebooks/tensorflow/Tensorflow2.0--rumendaoshijian/chapter_5"

os.chdir(path)
os.listdir(path)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['data',
 'tf_keras_to_estimator.ipynb',
 'baseline_model',
 'linear_model',
 'dnn_model',
 'tf_premade_estimators.ipynb',
 'tf_premade_estimators-new_feature.ipynb']

In [5]:
# https://storage.googleapis.com/tf-datasets/titanic/train.csv
# https://storage.googleapis.com/tf-datasets/titanic/eval.csv
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

print(train_df.head())
print(eval_df.head())

   survived     sex   age  ...     deck  embark_town  alone
0         0    male  22.0  ...  unknown  Southampton      n
1         1  female  38.0  ...        C    Cherbourg      n
2         1  female  26.0  ...  unknown  Southampton      y
3         1  female  35.0  ...        C  Southampton      n
4         0    male  28.0  ...  unknown   Queenstown      y

[5 rows x 10 columns]
   survived     sex   age  ...     deck  embark_town  alone
0         0    male  35.0  ...  unknown  Southampton      y
1         0    male  54.0  ...        E  Southampton      y
2         1  female  58.0  ...        C  Southampton      y
3         1  female  55.0  ...  unknown  Southampton      y
4         1    male  34.0  ...        D  Southampton      y

[5 rows x 10 columns]


In [6]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

print(train_df.head())
print(eval_df.head())
print(y_train.head())
print(y_eval.head())

      sex   age  n_siblings_spouses  parch  ...  class     deck  embark_town alone
0    male  22.0                   1      0  ...  Third  unknown  Southampton     n
1  female  38.0                   1      0  ...  First        C    Cherbourg     n
2  female  26.0                   0      0  ...  Third  unknown  Southampton     y
3  female  35.0                   1      0  ...  First        C  Southampton     n
4    male  28.0                   0      0  ...  Third  unknown   Queenstown     y

[5 rows x 9 columns]
      sex   age  n_siblings_spouses  parch  ...   class     deck  embark_town alone
0    male  35.0                   0      0  ...   Third  unknown  Southampton     y
1    male  54.0                   0      0  ...   First        E  Southampton     y
2  female  58.0                   0      0  ...   First        C  Southampton     y
3  female  55.0                   0      0  ...  Second  unknown  Southampton     y
4    male  34.0                   0      0  ...  Second     

In [7]:
train_df.describe()

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [21]:
def test_categorical_column_with_vocabulary_list():
  color_data = {'color': [['R', 'R'], ['G', 'R'], ['B', 'G'], ['A', 'A']]}  # 4行样本  
  color_column = tf.feature_column.categorical_column_with_vocabulary_list(  
      'color', ['R', 'G', 'B'], dtype=tf.string, default_value=-1  
  )  
  # 个人理解indicator_column就是先将每一个字符串表示成onehot编码
  # 比如R:100, G：010， B:001
  # ['R', 'R'] 就是100+100=200
  # ['G', 'R'] 就是010+100=110
  # ['B', 'G'] 就是001+010=100
  # ['A', 'A'] 就是000+000=000
  # 将稀疏的转换成dense，也就是one-hot形式，只是multi-hot  
  color_column_identy = tf.feature_column.indicator_column(color_column)  
  feature_layer = tf.keras.layers.DenseFeatures(color_column_identy)#根据处理后的一个特征构建网络层  
  print(feature_layer(color_data).numpy())#在网络层中输出预处理后的数据  

test_categorical_column_with_vocabulary_list()
#

[[2. 0. 0.]
 [1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 0.]]


In [22]:
def test_crossed_column():  
  """ crossed column测试 """  
  featrues = {  
   'price': [['A'], ['B'], ['C'],['A']],  
   'color': [['R'], ['G'], ['B'],['R']]  
  }  
  price = tf.feature_column.categorical_column_with_vocabulary_list('price', ['A', 'B', 'C', 'D'])  
  color = tf.feature_column.categorical_column_with_vocabulary_list('color', ['R', 'G', 'B'])  
  p_x_c = tf.feature_column.crossed_column([price, color], 16)#16是输出特征维度，crossed_column产生的特征依然是离散的  
  #特征交叉的时候形成一个新的特征[[AR],[BG],[CB],[AR]]
  #然后将特征[AR],[BG],[CB],[AR]映射到16维的onehot编码（其实12维就可以）
  #这样就形成了四个结果
  #[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  #[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  #[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  #[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
  p_x_c_identy = tf.feature_column.indicator_column(p_x_c)#将离散特征onehot  
  feature_layer = tf.keras.layers.DenseFeatures(p_x_c_identy)#根据处理后的一个特征构建网络层  
  print(feature_layer(featrues).numpy())#在网络层中输出预处理后的数据  
  
test_crossed_column()  

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [23]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class',
                       'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(categorical_column, vocab)
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                categorical_column, vocab)))

for numeric_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(
            numeric_column, dtype=tf.float32))

# cross feature: age: [1,2,3,4,5], gender:[male, female]
# 交叉特征是指对两个离散特征做笛卡尔积（感觉说的不对），比如说我的age有5个值，性别由2个值
# age_x_gender: [(1, male), (2, male), ..., (5, male), ..., (5, female)] #感觉这里说的不对，可能他说的应该是可以有这些值
# 假设age*gender有100000个值: 100 -> hash(100000 values) % 100 
# 同样的一个样本的特征都是离散特征的话，我们给他们全局的笛卡尔积的话就能记住每一条样本了
feature_columns.append(
    tf.feature_column.indicator_column( 
        tf.feature_column.crossed_column( #交叉特征API : tf.feature_column.crossed_column
            ['age', 'sex'], hash_bucket_size = 100))) #hash_bucket_size 是个整数
            #hash_bucket_size的物理含义是： 我们假设age*gender有10w个值，那我们一股脑的把这10w特征值都加到模型当中去
            #会大大增加模型的大小，相当于我们的input_layer呢直接从100变成了10w零100，这是十分不划算的，因为这10w个值呢
            #只有一个值是被激活的，他是一个非常稀疏的onehot编码，而hash_bucket_size就是用来控制这个值的，在这里设成了100
            #我对这10w值呢，都做一个hash，hash(100000 values) % 100，我就把这10w个值呢，映射到100个桶中去了，这样呢，就
            #减小了模型的size.比如说我原来有100个特征，加上这100个特征，也才只有200个特征而已，这是hash_bucket_size的物理含义
            #然后他为什么能起效果呢，在于这10w个值呢，是非常稀疏的，我让一些值呢进行了重用，把它缩成了100个，也就是说呢，这100个
            #有很多值的位置是一样的，但是呢因为他太稀疏了，我把这些值呢放到一个位置上也依然可以work
            #这样就得到一组交叉特征，对于linearClassifier可以直接输入进去，但是呢，dnnClassifier来说呢，必须把他转化成indicator_column
            #才能够输入进去(这话说的貌似不对，都要经过indicator_column吧)



sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [0]:
def make_dataset(data_df, label_df, epochs = 10, shuffle = True,
                 batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(data_df), label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [25]:
output_dir = 'baseline_model_new_features'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

baseline_estimator = tf.estimator.BaselineClassifier(
    model_dir = output_dir,
    n_classes = 2)
baseline_estimator.train(input_fn = lambda : make_dataset(
    train_df, y_train, epochs = 100))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'baseline_model_new_features', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating

FailedPreconditionError: ignored

In [26]:
baseline_estimator.evaluate(input_fn = lambda : make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False, batch_size = 20))

INFO:tensorflow:Could not find trained model in model_dir: baseline_model_new_features, running initialization to evaluate.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-05T03:58:13Z
INFO:tensorflow:Graph was finalized.


FailedPreconditionError: ignored

In [27]:
linear_output_dir = 'linear_model_new_features'
if not os.path.exists(linear_output_dir):
    os.mkdir(linear_output_dir)
linear_estimator = tf.estimator.LinearClassifier(
    model_dir = linear_output_dir,
    n_classes = 2,
    feature_columns = feature_columns)
linear_estimator.train(input_fn = lambda : make_dataset(
    train_df, y_train, epochs = 100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'linear_model_new_features', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this 

In [29]:
linear_estimator.evaluate(input_fn = lambda : make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-05T03:59:04Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from linear_model_new_features/model.ckpt-1960
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.50553s
INFO:tensorflow:Finished evaluation at 2020-04-05-03:59:04
INFO:tensorflow:Saving dict for global step 1960: accuracy = 0.7992424

{'accuracy': 0.79924244,
 'accuracy_baseline': 0.625,
 'auc': 0.853719,
 'auc_precision_recall': 0.76438755,
 'average_loss': 0.4587563,
 'global_step': 1960,
 'label/mean': 0.375,
 'loss': 0.44392708,
 'precision': 0.72115386,
 'prediction/mean': 0.39375186,
 'recall': 0.75757575}

In [30]:
dnn_output_dir = './dnn_model_new_features'
if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)
dnn_estimator = tf.estimator.DNNClassifier(
    model_dir = dnn_output_dir,
    n_classes = 2,
    feature_columns=feature_columns,
    hidden_units = [128, 128],
    activation_fn = tf.nn.relu,
    optimizer = 'Adam')
dnn_estimator.train(input_fn = lambda : make_dataset(
    train_df, y_train, epochs = 100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './dnn_model_new_features', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this w

In [31]:
dnn_estimator.evaluate(input_fn = lambda : make_dataset(
    eval_df, y_eval, epochs = 1, shuffle = False))
#同样的特征提取方法对于不同的模型效果是不一样的，所以我们在做特征工程的时候，不仅要考虑特征之间的相关性互补性
#还要考虑他和模型之间适配能力。

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-05T03:59:33Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./dnn_model_new_features/model.ckpt-1960
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.44355s
INFO:tensorflow:Finished evaluation at 2020-04-05-03:59:33
INFO:tensorflow:Saving dict for global step 1960: accuracy = 0.7462121,

{'accuracy': 0.7462121,
 'accuracy_baseline': 0.625,
 'auc': 0.8461279,
 'auc_precision_recall': 0.7779369,
 'average_loss': 0.69689786,
 'global_step': 1960,
 'label/mean': 0.375,
 'loss': 0.6545298,
 'precision': 0.625,
 'prediction/mean': 0.48599353,
 'recall': 0.8080808}